In [37]:
import pandas as pd
import scipy as sp
from scipy.sparse import diags
import numpy as np
from numpy import linalg as LA
import sys

from os import path

import matplotlib.pyplot as plt

#importing seaborn for plotting
import seaborn as sns

#for plotting purposes
%pylab inline
sns.set_style('ticks')
sns.set_context('paper')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib as mpl

from scipy.signal import find_peaks

# mpl.rcParams
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['figure.figsize'] = [8, 16/3]

Populating the interactive namespace from numpy and matplotlib


/Users/dosoonkim/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['number']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### converting ladder to nts

In [38]:
data_dir = './raw_data/'
fig_dir = './plots/'
processed_dir = './processed_data/'

In [44]:
#list of filenames 
filenames = ["Eukaryote Total RNA Nano_2020-11-24_11-30-28",\
             "Eukaryote Total RNA Nano_2020-11-24_11-37-12",\
             "Eukaryote Total RNA Nano_2020-11-24_11-55-27",\
             "Eukaryote Total RNA Nano_2020-11-24_12-02-25",\
             "Eukaryote Total RNA Nano_2020-11-24_12-16-31",\
             "Eukaryote Total RNA Nano_2020-11-24_12-22-23",\
             "Eukaryote Total RNA Nano_2020-11-24_13-04-00",\
             "Eukaryote Total RNA Nano_2020-11-24_13-12-23",\
             "Eukaryote Total RNA Nano_2020-11-24_13-30-10",\
             "Eukaryote Total RNA Nano_2020-11-24_13-37-35"]



for filename in filenames:
    
    """
    Plotting and identifying peaks from the ladder traces from BioAnalyzer.
    Must be done per chip/run, since ladders may elute at slightly different times per run.
    Generates plots that shows identified peaks.
    As of 11-30-2020, should identify 6 peaks.
    """
    
    ladder_dict = {}
    ladder_values = {}

    ladder_df = pd.read_csv(data_dir+filename+'_Ladder.csv', skiprows=17)[:-1]
    ladder_time = np.array(ladder_df['Time'].astype(float))
    ladder_value = np.array(ladder_df['Value'].astype(float))

    peaks,_ = find_peaks(ladder_value, height=5, distance=15)
    ladder_dict = list(peaks[:6])

    #plotting ladder trace + peak identification
    plot(ladder_time, ladder_value, label='ladder trace')
    plot(ladder_time[peaks], ladder_value[peaks], 'x', label='peaks', markersize=15)
    title('Ladder Peaks: '+filename, fontsize=16)
    legend(fontsize=14)
    xlabel('Time', fontsize=14)
    ylabel('FU', fontsize=14)
    xticks(fontsize=14)
    yticks(fontsize=14)
    tight_layout()
    
    #saving figure
    savefig(fig_dir+filename+'_ladder.png', dpi=300)
    clf()
    
    
    """
    Converting times (elution times from BioAnalyzer) to nucleotides using
    a 4th order polynomial fit
    """
    
    #returns the times when peaks occur, will serve as x on polynomial fit
    peak_times = ladder_time[ladder_dict]
    
    #ladder nucleotide sizes, will serve as y on polynomial fit
    peak_fu = np.array([25, 200, 500, 1000, 2000, 4000])
    
    #assert that the lengths of x and y match (cannot perform fit otherwise)
    assert (len(peak_times) == len(peak_fu))

    new_height = 5
    while (len(peak_times) < len(peak_fu)):
        new_height = new_height-0.1
        peaks,_ = find_peaks(ladder_value, height=new_height, distance=15)
        ladder_dict = list(peaks[:6])
        peak_times = ladder_time[ladder_dict]
        
    #fitting a 4th order polynomial to peaks/nucleotides
    ladder_fit = np.polyfit(x=peak_times, y = peak_fu, deg = 4)
    lf = np.poly1d(ladder_fit)
    ladder_nts = lf
    
    #plotting
    plot(peak_fu, lf(peak_times), 'o',label='calculated nts vs. ladder nts')
    plot(peak_fu, peak_fu,label='perfect correlation')
    ylabel('Calculated nts (from time)', fontsize=14)
    xlabel('Ladder nts', fontsize=14)
    yticks(fontsize=12)
    xticks(fontsize=12)
    title('Ladder polynomial fit: '+filename, fontsize=16)
    legend(fontsize=12)
    tight_layout()
    
    #saving figure
    savefig(fig_dir+'ladder_fit-'+filename+'.png', dpi=300)
    clf()

    
    """
    Applying fits to samples as well, per ladder
    """
    
    sample_numbers = np.arange(1,13)

    for number in sample_numbers:
        file = filename+'_Sample'+str(number)+".csv"
        print(file)
        if (path.exists(data_dir + file)):
            sample_df = pd.read_csv(data_dir + file, skiprows=17)[:-1]
            times = np.array(sample_df['Time'].astype(float))
            sample_df['Nucleotides'] = ladder_nts(times)
            sample_df.to_csv(processed_dir+'nts-'+ file)
        else:
            continue

'\n    Plotting and identifying peaks from the ladder traces from BioAnalyzer.\n    Must be done per chip/run, since ladders may elute at slightly different times per run.\n    Generates plots that shows identified peaks.\n    As of 11-30-2020, should identify 6 peaks.\n    '

Text(0.5, 1.0, 'Ladder Peaks: Eukaryote Total RNA Nano_2020-11-24_11-30-28')

Text(0.5, 0, 'Time')

Text(0, 0.5, 'FU')

(array([10., 20., 30., 40., 50., 60., 70., 80.]),
 <a list of 8 Text major ticklabel objects>)

...

(array([-500.,    0.,  500., 1000., 1500., 2000., 2500., 3000., 3500.,
        4000., 4500.]),
 <a list of 11 Text major ticklabel objects>)

Text(0.5, 1.0, 'Ladder polynomial fit: Eukaryote Total RNA Nano_2020-11-24_13-37-35')

'\n    Applying fits to samples as well, per ladder\n    '

Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample1.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample2.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample3.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample4.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample5.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample6.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample7.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample8.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample9.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample10.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample11.csv
Eukaryote Total RNA Nano_2020-11-24_13-37-35_Sample12.csv


<Figure size 576x384 with 0 Axes>